# Llama 2: Prompt Engineering Expirements to Extract Information

This notebook documents expirements with Llama 2 prompts. 

In [3]:
import requests, time, os, json
from dotenv import load_dotenv
from IPython.display import display, HTML

In [14]:
# To use Llama 2 70B on HuggingFace requires an authentication token and HuggingFace Pro account that cost $9 a month.  
# To learn more see 
# - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf?inference_api=true
# - https://huggingface.co/pricing

# Loading authentication token from .env file
load_dotenv('../.env')
together_token = os.getenv("TOGETHER_TOEKN")


### Generalize methods and class that will be used in the expirements below

In [13]:
# Object to represent an answer from Llama
class Answer:
    def __init__(self, answer, elapse):
        self.answer = answer
        self.elapse = elapse


In [11]:
# MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1"  #  "togethercomputer/llama-2-13b-chat"

def together_generate(prompt, temperature = 0.2, top_p = 0.8, top_k = 70) -> str:

    URL = "https://api.together.xyz/inference"
    
    #  "stop": ".",
    payload = {
        "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
        "prompt": prompt,
        "max_tokens": 1000,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "repetition_penalty": 1,
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {together_token}",
        "User-Agent": "Acme Benchmark",
    }

    response = requests.post(URL, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        print(response.text)
        return response.text


In [15]:
# Wrapper function that run generate and return an Answer
def run_prompt(prompt: str, temperature = 0.2, top_p = 0.8, top_k = 70) -> Answer:
    start_time = time.time()
    res = together_generate(prompt)
    answer = res['output']['choices'][0]['text']
    end_time = time.time()
    elapse = round(end_time - start_time)
    return Answer(answer, elapse)

In [16]:
# Display answer object in HTML
def display_answer(answer: Answer, header = ''):
    answer_html_template = """<h3>{HEADER} Answer - Time to Generate: {ELAPSE} seconds</h3>
    <textarea cols='100' rows={NUM_ROWS}>{ANSWER}</textarea>"""
    
    number_rows = (len(answer.answer.split(' ')) / 10)
    
    html = answer_html_template.format(ANSWER=answer.answer, ELAPSE=answer.elapse, HEADER=header, NUM_ROWS=number_rows)
    display(HTML(html))

## The Expirements Focus
The expirements will be focused on extracting information from a blogpost from Addresson Horoviz about mobile games soft launch. Since the Llama have size limit of tokens a subset of the post is used.  

In [7]:
cwd = os.getcwd()
cwd = cwd.replace('/notebooks', '')
directory = os.path.join(cwd, "tests/data/")

def read_txt_files(directory):
    txt_files = []
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            file_path = os.path.join(directory, file)
            with open(file_path, "r") as f:
                txt_files.append(f.read())
    return txt_files

text_files = read_txt_files(directory)

In [8]:
text = text_files[0]

In [9]:
p1 =  """Write a concise summary of the main ideas in article below in bullet-points, don't repeat ideas. article: {BODY}""".format(BODY=text)

display_answer(run_prompt(p1))

In [32]:
p2 =  """How would you categorized the following text? Is it news article, blog, research paper? 
Answer JSON following this format: {{'category': 'news', 'explanation': 'text of the article'}}.
Text: {BODY}
JSON:""".format(BODY=text)

display_answer(run_prompt(p2))

In [ ]:
p3 =  """Summarizes the article in multiple bullet-points. The number of bullet points must below six. article: {BODY}""".format(BODY=text)
p3_answer = run_prompt(p3)
display_answer(p3_answer)

Not bed, Llama listen to me :)

# Expirement 2: System Message

The Llama paper describe the system message that uses to set the stage and concext for the model. 
In the following example, I am using the system messsage. Let see what is the different between P3 that doesn't have system message and P4 that use system message.   

In [17]:
p4 = """<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write a concise TL;DR summary in numeric bullet-points for the following article. Only include bullet-points. 
Limit the number of bullet-point to five. Output the answer in JSON format [{{"1", bullet-point}}, {{"2", bullet-point}}, ...]

article: {BODY}""".format(BODY=text)

# p4_answer = run_prompt(p4)
# display_answer(p4_answer, "P4 - With System Message")
# Display p3 too for comparision. 

display_answer(run_prompt(p4))


Both P3 and P4 are pretty good and it's hard to see the different the the system message added. I personally prefer P4 (system message) because the answer read a better in my opinion, but I am sure someone will argue with on that. 

# Expirence 3: Modify the System Message
The system message can be modified to better fit to the task and define the persona and context we want Llama to assume. 

Changes applied to the original system message:
- Use the researcher persona and specify the tasks to summarizing articles. 
- Remove safety instruction, there are not needed since we asking Llama to be truthful to the article. 

In [ ]:
p5 = """<s>[INST] <<SYS>>
You are a researcher task in summarizing and writing concise brief of articles.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.
<</SYS>>
In one sentence, tell me what is the main idea of the following article. Limit the answer to tweenty words. 
Article: {BODY}
[/INST]""".format(BODY=text)

display_answer(run_prompt(p5))



The answer for p5 is the best in my opinion so far. I like the into and conclusion that Llama addeed. 

# Expirement 4: Asking Questions about the Article

The article is about 'Mobile Game Soft Launch' let ask Llama specific question about it. The answer is pretty good. 

You are a researcher task with answering questions about an article.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.

In [18]:
p6 = """[INST]
You are a researcher task with extracting information from articles.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.

According to the article mobile game soft launch solves what problems? Please include examples from the article in your answer. 
Answer should include at leat 50 words.
Article: {BODY}
[/INST]
Answer:
""".format(BODY=text)

display_answer(run_prompt(p6))


This is pretty good, let see if we improve on that. By asking Llama what is the article is about and then use the answer to ask additional questions. 
Prompt 7, asks Llama to tell what the article is about and then the answer is used to generate a prompt 8 that ask a second question.

To make it easy to programmatically use the answer, I asked Llama to output the answer in JSON. Using expirements (that I didn't included here) I descover that Llama need a template and being told to only output valid JSON. 

In [19]:
p7 = """<s>[INST] <<SYS>>
You are a researcher task with answering questions about an article.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.
<</SYS>>
Tell me what is the article about in one to three words? 
Output the answer in JSON in the following format {{"article_is_about": answer}}. Only output JSON
Article: {BODY}
[/INST]""".format(BODY=text)

a7 = run_prompt(p7)
json_a7 = json.loads(a7.answer)
print(f"JSON Answer:\n {json_a7}")
about = json_a7['article_is_about']
print(about)


p8 = """<s>[INST] <<SYS>>
You are a researcher task with answering questions about an article.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.
<</SYS>>
According to the article, what problems does {{ABOUT}} solves? Give several short examples from the article in your answer. Limit the answer to fifty words.
Article: {BODY}
Answer:
[/INST]""".format(BODY=text, ABOUT=about)

display_answer(run_prompt(p8),'Prompt 8')


JSON Answer:
 {'article_is_about': "Richard Romanus' Death"}
Richard Romanus' Death


Wow this is awesume. We can feed answers into new prompts to refine the information we try to extract. 

Let try a different question. What industry the article is about.

In [ ]:
p9 = """<s>[INST] <<SYS>>
You are a researcher task with answering questions about an article.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.
<</SYS>>
Name the industry the article is focus on? Output only the industry name.
Article: {BODY}
[/INST]""".format(BODY=text, ABOUT=about)

display_answer(run_prompt(p9))

Notice that I asked the answer to include only the industry name, but Llama disregarded my request and wrote a sentence. 
Let see if we can fixed that by asking the answer to be in JSON. It worked!!! 

Note: I needed to add "include only valid JSON" to prevent Llama then adding an explanation.  

In [ ]:
p10 = """<s>[INST] <<SYS>>
You are a researcher task with answering questions about an article.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.
<</SYS>>
Name industry the article is focus on? Output only the industry name. Output the answer in JSON, using format {{"industry": industry}}.
Include only valid JSON.
Article: {BODY}
[/INST]""".format(BODY=text)

display_answer(run_prompt(p10))

Let try and trick Llama and ask him what sport is the article focuses on? 

In [ ]:
p11 = """[INST]
You are a researcher task with answering questions about an article.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.

Name sport the article is focus on? Output only the sport name. Output the answer in JSON, using format {{"sport": sport, "explanation": explanation}}. 
Include only valid JSON. Make sure to close the JSON object with a curly bracket.
Article: {BODY}
[/INST]""".format(BODY=text)

display_answer(run_prompt(p11))

In [ ]:
p12 = """<s>[INST] <<SYS>>
You are a researcher task with answering questions about an article.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.

Output answer in JSON using the following format: {{"name": name, "type": type, "explanation": explanation}}
<</SYS>>

What entities mentioned in the article that can generalize the topic? [/INST] 
[
{{"name": "semiconductor", "type": "industry", "explanation": "Companies engaged in the design and fabrication of semiconductors and semiconductor devices"}},
{{"name": "NBA", "type": "sport league", "explanation": "NBA is the national basketball league"}},
{{"name": "Ford F150", "type": "vehicle", "explanation": "Article talks about the Ford F150 truck"}},
] </s>

<s>[INST]   
What entities mentioned are important to the article subject? Limit the answert to ten most important entities. 
Output answer in JSON using the following format: {{"name": name, "type": type, "explanation": explanation}}
Article: {BODY}
[/INST]""".format(BODY=text)

display_answer(run_prompt(p12))

In [20]:
p13 = """<s>[INST] <<SYS>>
You are a researcher task with answering questions about an article.  
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer, please don't share false information.

<</SYS>>

Answers output must confirm to the this JSON format [/INST] 

JSON Output: {{
"oneSentenceSummary" : "Mobile game soft launch is a process of releasing a game to a limited audience for testing.",
"summaryInNumericBulletPoints" : [
"1. Mobile game soft launch is a process of releasing a game to a limited audience for testing.",
"2. Mobile game soft launch is a process of releasing a game to a limited audience for testing.",
]
"entities : [
{{"name": "semiconductor", "type": "industry", "explanation": "Companies engaged in the design and fabrication of semiconductors and semiconductor devices"}},
{{"name": "NBA", "type": "sport league", "explanation": "NBA is the national basketball league"}},
{{"name": "Ford F150", "type": "vehicle", "explanation": "Article talks about the Ford F150 truck"}},
]
}} </s>

<s>[INST]
Use the examples above to answer the following question.
1. Summarize the article in one sentence. Limit the answer to twenty words.
2. Summarize the article in multiple bullet-points. Each bullet-point need to have betweeen ten to tweenty words. Limit the number of bullet points must below six.
3. Identify ten entities (companies, people, location, products....) mentioned in the article. Include short explanation for each entity.

Use the JSON format above to output your answer. Only output valid JSON format.
Article: {BODY}
[/INST]""".format(BODY=text)

display_answer(run_prompt(p13))

In [ ]:
directory = "~/Projects/icognition-backend/tests/data/"

def read_txt_files(directory):
    txt_files = []
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            file_path = os.path.join(directory, file)
            with open(file_path, "r") as f:
                txt_files.append(f.read())
    return txt_files